In [2]:
import pandas as pd
import numpy as np

import os

#os.listdir(".")
#credit = pd.read_csv("Credit_DataSet.csv")

# TODO: READ THE PROCESSED CSV INSTEAD (FOR EXAMPLE WITH CONVERTED CURRENCY)!

credit = pd.read_excel("./Credit/Credit_DataSet.xlsx", sheetname="DATA", skiprows=0)

# Quickly select numeric columns

to_keep = ['NBR_EMPLOYEES', 'COUNTRY_RISK_GROWTH_RATE',
       'COUNTRY_RISK_DIVIDEND_YIELD', 'COUNTRY_RISK_PAYOUT_RATIO',
       'VOLATILITY_30D', 'VOLATILITY_180D', 'PCT_CHG_1_YEAR', 'PCT_CHG_6_M',
       'DIVIDEND_YIELD', 'MARKETCAP', 'TOTAL_ASSETS', 'TOTAL_LIABILITIES',
       'CURRENT_ASSETS', 'EBIT', 'RETAINED_EARNINGS', 'SALES', 'SALES_GROWTH',
       'INTEREST_EXPENSES']

X = credit[ to_keep + ["DEFAULT_PROB"] ].dropna()

Y = X["DEFAULT_PROB"].values
X = X[to_keep].values

In [19]:
# READ PREPROCESSED DATA FROM R FOR NN
X_df = pd.read_csv("some_X.csv", usecols=['NBR_EMPLOYEES', 'EMPL_GROWTH',
       'COUNTRY_RISK_GROWTH_RATE', 'COUNTRY_RISK_DIVIDEND_YIELD',
       'COUNTRY_RISK_PAYOUT_RATIO', 'VOLATILITY_30D', 'VOLATILITY_180D',
       'PCT_CHG_1_YEAR', 'PCT_CHG_6_M', 'DIVIDEND_YIELD', 'MARKETCAP',
       'TOTAL_ASSETS', 'TOTAL_LIABILITIES', 'CURRENT_ASSETS', 'EBIT',
       'RETAINED_EARNINGS', 'SALES', 'SALES_GROWTH', 'INTEREST_EXPENSES',
        'WHISTLE_BLOWER_POLICY', 'ETHICS_POLICY', 'BRIBERY_POLICY'])
Y_df = pd.read_csv("some_Y.csv", usecols=[1])

import torch
from torch import nn
from torch.autograd import Variable

X = Variable(torch.FloatTensor(X_df.values)).cuda()
Y = Variable(torch.FloatTensor(Y_df.values)).cuda() # Always stays the same

In [101]:
def select_most_likely(mlp, loss_fn, x_row, y_row):
    candidates = [] # TODO: For every -1 in row in the specific columns, try 0 and 1
    
    bad_values = {'WHISTLE_BLOWER_POLICY': 0,
                            'ETHICS_POLICY': 0,
                            'BRIBERY_POLICY': 0}

    possible_values_dict = {'WHISTLE_BLOWER_POLICY':[[1,0], [0,1]],
                            'ETHICS_POLICY':[[1,0], [0,1]],
                            'BRIBERY_POLICY':[[1,0], [0,1]]}
    
    # Make a dictionary for the candidate values that we will expand later
    candidate_dict = {}
    for k, v in x_row.iteritems():
        candidate_dict[k] = []
        
        if (k in bad_values): # We have to encode for the neural network
            if (v == bad_values[k]):
                for i in possible_values_dict[k]:
                    # Directly encode the bad_value
                    candidate_dict[k].append(i)
            else:
                candidate_dict[k] = [possible_values_dict[k][int(v) - 1]] # HACK: IT'S BECAUSE THEY HAVE VALUES 1 AND 2
        else:
            # Just add the value
            candidate_dict[k].append([v])

    best_candidate = None
    lowest_loss = 100000000000

    # In the dictionary we have the different candidates
    for candidate in product(*candidate_dict.values()):
        input = torch.FloatTensor(np.array(list(chain(*list(candidate))))).cuda()
        print(len(input))
        # Forward the candidate through the network
        y_pred = mlp(input)
        loss = loss_fn(y_pred, y_row)
        if loss < lowest_loss:
            best_candidate = input
    return best_candidate
    
def get_E_step(mlp, loss_fn, X, Y):
    df_list = []
    for x_row, y_row in zip(X.iterrorws(), Y.iterrows()):
        df_list.append(select_most_likely(mlp, loss_fn, x_row, y_row))

    return pd.DataFrame(df_list)


In [105]:
(i, bla) = list(X_df.iterrows())[100]
(j, blu) = list(Y_df.iterrows())[100]
print(bla.keys())


from itertools import * 

#select_most_likely(model, loss_fn, bla, 1)
select_most_likely(model, loss_fn, bla, 1)


Index(['NBR_EMPLOYEES', 'EMPL_GROWTH', 'COUNTRY_RISK_GROWTH_RATE',
       'COUNTRY_RISK_DIVIDEND_YIELD', 'COUNTRY_RISK_PAYOUT_RATIO',
       'VOLATILITY_30D', 'VOLATILITY_180D', 'PCT_CHG_1_YEAR', 'PCT_CHG_6_M',
       'DIVIDEND_YIELD', 'MARKETCAP', 'TOTAL_ASSETS', 'TOTAL_LIABILITIES',
       'CURRENT_ASSETS', 'EBIT', 'RETAINED_EARNINGS', 'SALES', 'SALES_GROWTH',
       'INTEREST_EXPENSES', 'WHISTLE_BLOWER_POLICY', 'ETHICS_POLICY',
       'BRIBERY_POLICY'],
      dtype='object')
25


TypeError: torch.mm received an invalid combination of arguments - got (torch.cuda.FloatTensor, Variable), but expected one of:
 * (torch.cuda.FloatTensor source, torch.cuda.FloatTensor mat2)
      didn't match because some of the arguments have invalid types: ([32;1mtorch.cuda.FloatTensor[0m, [31;1mVariable[0m)
 * (torch.cuda.sparse.FloatTensor source, torch.cuda.FloatTensor mat2)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.cuda.FloatTensor[0m, [31;1mVariable[0m)


In [69]:
list(chain(*[0.12131207243014801, -0.0395071823577576, 2.48383457084033, -0.0992536745005148, -0.47539842454617204, -0.609546912466737, -0.872210990613752, -0.5390015136315179, -0.6930698997713771, 0.7908406097201909, -0.00038096724263901, 0.0442511849665407, 0.0262971823839764, 0.0140160540741605, -0.00043371208712998703, 0.181652875331318, -0.156166111876235, -0.0641558014418736, 0.5107643081367039, 1, [1, 0], [1, 0]]))

TypeError: 'float' object is not iterable

In [23]:
def get_M_step(mlp, loss_fn, X, Y):
    learning_rate = 1e-4
    for t in range(N_iter):
        y_pred = model(X)
        loss = loss_fn(y_pred, Y)
        if (t % (N_iter / 10) == 0): print(loss)
        model.zero_grad()
        loss.backward()
        for param in model.parameters():
            param.data -= learning_rate * param.grad.data
    return mlp

def EM_algorithm(X, Y):
    # Make an initial model
    N_hidden = 5
    N_iter = 50000
    model = torch.nn.Sequential(
        torch.nn.Linear(X.shape[1] + 3, N_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(N_hidden, Y.shape[1]),
        torch.nn.Sigmoid() # If it doesn't work, look here
    )

    model = model.cuda()
    loss_fn = torch.nn.SoftMarginLoss(size_average=False)
    
    # Init
    for i in range(100):
        # Get the expected data
        print("Doing iteration {} E step".format(i))
        X = Variable(torch.FloatTensor(get_E_step(model, loss_fn, X_df, Y_df).values)).cuda()
        # TODO: Check if the predicted data is the same still?
        # Train the model for some iterations
        model = get_M_step(mlp, loss_fn, X, Y)
        print("Doing iteration {} M step".format(i))
    return X, mlp
    

In [95]:
    N_hidden = 5
    N_iter = 50000
    model = torch.nn.Sequential(
        torch.nn.Linear(X.shape[1] + 3, N_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(N_hidden, Y.shape[1]),
        torch.nn.Sigmoid() # If it doesn't work, look here
    )

    model = model.cuda()
    loss_fn = torch.nn.SoftMarginLoss(size_average=False)


25